In [5]:
import pennylane as qml
from tqdm import tqdm

nqubits = 25
dev = qml.device("lightning.gpu", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(a, wires=range(nqubits), pad_with=0, normalize=True)

    qml.adjoint(qml.AmplitudeEmbedding(b, wires=range(nqubits), pad_with=0, normalize=True))
    
    return qml.probs(wires=range(nqubits))

from sklearn.svm import SVC
def qkernel(A, B):
    K = np.zeros((len(A), len(B)))
    for i, a in tqdm(enumerate(A), total=len(A)):
        for j, b in enumerate(B):
            if j < i:
                K[i, j] = K[j, i]
                continue
            K[i, j] = kernel_circ(a, b)[0]
        print(f"  Finished row {i+1}")
    return K


In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.decomposition import PCA

df = pd.read_csv('pd_speech_features.csv', header=1)


selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(df.drop(columns=['id', 'gender', 'class']))

scaler = StandardScaler()
scaled_features = scaler.fit_transform(selected_features)

pca_scaled = PCA(n_components=150, svd_solver='full')
pca_scaled.fit(scaled_features)

variance_scaled = pca_scaled.explained_variance_ratio_.cumsum()


transformed_scaled = pca_scaled.transform(scaled_features)

df_transformed_scaled = pd.DataFrame(data=transformed_scaled, columns=[f"PC{i+1}" for i in range(transformed_scaled.shape[1])])


X, y = pd.concat([df[["gender"]], df_transformed_scaled], axis=1), df["class"]

In [8]:
svm = SVC(kernel=qkernel)

In [9]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=15)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=15)
X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = X_test.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

In [ ]:
svm.fit(X_train, y_train)

  0%|          | 0/453 [00:00<?, ?it/s]